In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import requests
import pickle
import sys
import re
import csv

In [ ]:
crawled_liste = []

In [ ]:
class Article:
    def __init__(self, title, lead, datum, zeit, author, text, url):
        self.title = title
        self.lead = lead
        self.datum = datum
        self.zeit = zeit
        self.author = author
        self.text = text
        self.url = url

In [ ]:
driver = webdriver.Firefox()
driver.implicitly_wait(30)
driver.maximize_window()

In [ ]:
start_seite = "https://www.luzernerzeitung.ch/"
driver.get(start_seite)

cookies = pickle.load(open("nlz_cookies.pkl", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)
    
driver.refresh()

In [ ]:
with open("nlz_output.bin", "rb") as data:
    ueb_pages = pickle.load(data)

In [ ]:
urls = []
for element in ueb_pages:
    urlf = "https://www.luzernerzeitung.ch" + element
    urls.append(urlf)

In [ ]:
def inhalter():
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    try:
        title = soup.find(class_="headline__title")
        title = title.get_text() 
        title = title.replace("\n", " ")
        title = title.strip()
    except AttributeError:
        print("AttributError")
        pass
    print(title)

    try:
        lead = soup.find(class_="headline__lead").get_text()
        lead = lead.replace("\n", " ")
    except AttributeError:
        lead = ""

    try:
        author = soup.find(class_="metainfo__item metainfo__item--author").get_text()
    except AttributeError:
        author = ""


    try:
        time_indy = soup.find(class_="metainfo__item metainfo__item--date").get_text().strip()
        time_indy = time_indy.split(",")
        time = time_indy[1].strip()
        if "Uhr" in time:
            time = time.replace("Uhr", "")
        date = time_indy[0]
    except AttributeError:
        time = ""
        date = ""

    texte = soup.find_all("p", class_=re.compile("articlecomponent.text"))
    artikel = []
    for element in texte:
        element = element.get_text()
        artikel.append(element)

    article = " ".join(artikel)
    
    durl = page
    
    crawled = Article(title, lead, date, time, author, article, durl)
    crawled_liste.append(crawled)

In [ ]:
for page in urls:
    time.sleep(2)
    try:
        driver.get(page)
        inhalter()
    except TimeoutException:
        print("TimeOut")
        pass

In [ ]:
with open('nlz_art1.csv', 'w', newline='', encoding="utf8") as csvfile:
    blogwriter = csv.writer(csvfile, delimiter='|', quotechar='"')


    for article in crawled_liste:
        blogwriter.writerow( [article.title, article.lead, article.datum, article.zeit, article.author, article.text, article.url] )